In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50, ResNet101, ResNet152
from tqdm import tqdm

In [2]:
SEED = 42
EPOCHS = 50
BATCH_SIZE = 32 
IMG_SIZE = 256
ROOT = 'flower_images/'

df = pd.read_csv(ROOT + 'flower_labels.csv')


In [17]:
df.tail(10)

,file,label
200,0201.png,rose
201,0202.png,iris
202,0203.png,peony
203,0204.png,bellflower
204,0205.png,leucanthemum maximum
205,0206.png,viola
206,0207.png,phlox
207,0208.png,leucanthemum maximum
208,0209.png,viola
209,0210.png,rose


In [3]:
df = df.replace({0:'phlox',1:'rose',2:'calendula',3:'iris',4:'leucanthemum maximum',
                 5:'bellflower',6:'viola',7:'rudbeckia laciniata',
                 8:'peony',9:'aquilegia'})


In [14]:
df.label.value_counts()

peony                   26
bellflower              25
viola                   23
iris                    22
leucanthemum maximum    21
phlox                   21
rose                    20
calendula               19
aquilegia               18
rudbeckia laciniata     15
Name: label, dtype: int64

In [4]:
train_df, test_df = train_test_split(df, 
                                     test_size=0.5, 
                                     random_state=SEED, 
                                     stratify=df['label'].values)



def create_datasets(df, img_size):
    imgs = []
    for file in tqdm(df['file']):
        img = cv2.imread(ROOT+file)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        imgs.append(img)
    # not normalize    
    imgs = np.array(imgs)
    df = pd.get_dummies(df['label'])
    return imgs, df


train_imgs, train_df = create_datasets(train_df, IMG_SIZE)
test_imgs, test_df = create_datasets(test_df, IMG_SIZE)

100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:01<00:00, 54.05it/s]


In [5]:
num_classes = len(df.label.value_counts())
def build_model(ResNet, img_size, n):
    inp = Input(shape=(img_size,img_size, n))
    resnet = ResNet(input_shape=(img_size,img_size,n),
                    weights='imagenet',
                    include_top=False)
    # freeze ResNet
    resnet.trainable = False
    x = resnet(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


resnet50 = build_model(ResNet50, IMG_SIZE, 3)
resnet50.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
train_df

,aquilegia,bellflower,calendula,iris,leucanthemum maximum,peony,phlox,rose,rudbeckia laciniata,viola
30,0,0,0,1,0,0,0,0,0,0
114,0,1,0,0,0,0,0,0,0,0
23,0,0,0,0,1,0,0,0,0,0
85,0,1,0,0,0,0,0,0,0,0
56,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
209,0,0,0,0,0,0,0,1,0,0
57,0,0,0,0,0,1,0,0,0,0
28,0,0,0,0,0,0,0,0,0,1
37,1,0,0,0,0,0,0,0,0,0


In [23]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('resnet50.h5', 
                                                monitor='loss', 
                                                save_best_only=True,
                                                save_weights_only=True)

resnet50.fit(train_imgs, train_df, batch_size=BATCH_SIZE,
          epochs=10, verbose=0, callbacks=[checkpoint])
resnet50.load_weights('resnet50.h5')


resnet50.evaluate(test_imgs, test_df)

4/4 [==============================] - 10s 2s/step - loss: 0.4834 - accuracy: 0.8952


[0.483414888381958, 0.8952381014823914]

In [25]:
from tensorflow.keras.models import load_model

resnet50.save('model_resnet50.h5')

In [18]:
#using resnet101
resnet101 = build_model(ResNet101, IMG_SIZE, 3)
resnet101.summary()


171450368/171446536 [==============================] - 52s 0us/step
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet101 (Functional)       (None, 8, 8, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 42,678,666
Trainable params: 20,490
Non-trainable params: 42,658,176
_________________________________________________________________


In [20]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('resnet101.h5', 
                                                monitor='loss', 
                                                save_best_only=True,
                                                save_weights_only=True)

resnet101.fit(train_imgs, train_df, batch_size=BATCH_SIZE,
              epochs=10, verbose=0, callbacks=[checkpoint])
resnet101.load_weights('resnet101.h5')

resnet101.evaluate(test_imgs, test_df)

4/4 [==============================] - 17s 4s/step - loss: 0.8619 - accuracy: 0.8095


[0.8619145154953003, 0.8095238208770752]

In [21]:
#usingresnet152
resnet152 = build_model(ResNet152, IMG_SIZE, 3)
resnet152.summary()

234700800/234698864 [==============================] - 83s 0us/step
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet152 (Functional)       (None, 8, 8, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 58,391,434
Trainable params: 20,490
Non-trainable params: 58,370,944
_________________________________________________________________


In [24]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('resnet152.h5', 
                                                monitor='loss', 
                                                save_best_only=True,
                                                save_weights_only=True)

resnet152.fit(train_imgs, train_df, batch_size=BATCH_SIZE,
              epochs=10, verbose=0, callbacks=[checkpoint])
resnet152.load_weights('resnet152.h5')

resnet152.evaluate(test_imgs, test_df)

4/4 [==============================] - 25s 6s/step - loss: 0.4489 - accuracy: 0.8667


[0.4488743245601654, 0.8666666746139526]